In [10]:
import plotly.express as px
import pandas as pd
import numpy as np

In [41]:
df = pd.read_csv('output.csv')
jitter = []
for tag in df['tag'].unique():
    jitter.append({
        'tag': tag,
        'jitter': np.random.uniform(-1, 1)
    })
jitter = pd.DataFrame(jitter)
df = df.merge(jitter, on='tag')

px.scatter(
    df, x="position", y="jitter", size="weight", color="reserves", hover_name="tag",
    range_x=[-0.1,1.1], range_y=[-1.1,1.1],
    animation_frame="time", animation_group="tag"
)

In [42]:
px.scatter(
    df, x="position", y="jitter", size="weight", color="win", hover_name="tag",
    range_x=[-0.1,1.1], range_y=[-1.1,1.1],
    animation_frame="time", animation_group="tag"
)

In [43]:
x = np.arange(0, 1.1, 0.001)
y = x ** 2
px.line(x=x, y=y).show()